In [17]:
import mysql.connector
import csv
import pandas as pd
import os
import tempfile
import re

In [18]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [19]:
con = mysql.connector.connect(user='andr',
                              password='rstq!2Ro',
                              host='127.0.0.1',
                              database='cat_all_domains',
                              auth_plugin='mysql_native_password'
                             )

In [20]:
curA = con.cursor(dictionary=True, buffered=True)
curB = con.cursor(dictionary=True, buffered=True)
curC = con.cursor(dictionary=True, buffered=True)
curD = con.cursor(dictionary=True, buffered=True)

## First experiments
curA and curB shows the first few experiments on morphology, where we were not keeping track of the POS.

In [21]:
curA.execute("SELECT id_unigram, unigram, freq_all, morph FROM unigrams;")
rows = curA.fetchall()

In [22]:
with open('morph_tags.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['id_unigram', 'unigram', 'freq_all', 'morph'])
    for dictionary in rows:
        writer.writerow([dictionary['id_unigram'], dictionary['unigram'], dictionary['freq_all'], dictionary['morph']])

df = pd.read_csv('morph_tags.csv')

df.head()

,id_unigram,unigram,freq_all,morph
0,36215,NUM,405252,_
1,47683,<URL>,1367,_
2,381939,А,2,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing
3,381940,В,2571,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing
4,381941,Малько,45,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing


In [23]:
curB.execute("SELECT morph, freq_all*count AS 'par_count' FROM (SELECT morph, freq_all, COUNT(*) AS 'count' FROM unigrams GROUP BY morph, freq_all) AS tab1")

In [24]:
rowsB = curB.fetchall()

In [25]:
with open('stats.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['morph', 'count'])
    for dictionary in rowsB:
        writer.writerow([dictionary['morph'], dictionary['par_count']])

In [26]:
df_par_stats = pd.read_csv('stats.csv')

df_par_stats[:10]

,morph,count
0,_,405252
1,_,1367
2,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,2474
3,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,2571
4,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,90
5,_,1013830
6,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,188
7,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,400
8,_,622277
9,Case=Nom|Degree=Pos|Number=Plur,600


## MySQL --> POS/tagset counts

With this MySQL search we extract from the database the frequency of use of each udpipe morphological tagset, preserving information about POS for each tagset. The aim is keeping a distinction between the different tagsets also based on the part of speech, because we assume POS behavior as disinct one from the other. 

In [27]:
curC.execute("""SELECT morph, pos, freq_all FROM
(SELECT morph, lemma, freq_all FROM unigrams) AS a JOIN
(SELECT id_lemmas, id_pos FROM lemmas) AS b ON lemma = id_lemmas JOIN pos ON b.id_pos = pos.id_pos;""")

In [28]:
rowsC = curC.fetchall()

In [29]:
with open('statistica.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['morph', 'POS', 'count'])
    for dictionary in rowsC:
        writer.writerow([dictionary['morph'], dictionary['pos'], dictionary['freq_all']])

In [30]:
df_statistica = pd.read_csv('statistica.csv')

df_statistica[:5]

,morph,POS,count
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,50
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,6
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,49
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,8
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,13


In [31]:
len(df_statistica)

700282

In [32]:
df_statistica = df_statistica[df_statistica.morph != '_']

In [33]:
len(df_statistica)

673942

In [34]:
keys = zip(df_statistica['morph'], df_statistica['POS'], df_statistica['count'])
# freq = df_statistica['count']

In [35]:
dictn = {}
for key in keys:
    if (key[0], key[1]) in dictn:
        dictn[(key[0], key[1])] += key[2]
    else:
        dictn[(key[0], key[1])] = key[2]

In [36]:
len(dictn)

663

In [37]:
df_statistica = pd.Series(dictn).reset_index()
df_statistica.columns = ['tagset', 'POS', 'count'] 
df_statistica[:5]

,tagset,POS,count
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,108669
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,94457
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,240823
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,133862
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,44188


In [38]:
df_statistica['part_freq'] = df_statistica['count'].apply(lambda x: x/df_statistica['count'].sum(axis=0))

In [39]:
df_statistica[:5]

,tagset,POS,count,part_freq
0,Case=Nom|Degree=Pos|Number=Plur,ADJ,108669,0.012420
1,Case=Nom|Degree=Pos|Gender=Masc|Number=Sing,ADJ,94457,0.010796
2,Case=Gen|Degree=Pos|Number=Plur,ADJ,240823,0.027524
3,Case=Gen|Degree=Pos|Gender=Masc|Number=Sing,ADJ,133862,0.015299
4,Case=Loc|Degree=Pos|Number=Plur,ADJ,44188,0.005050


In [40]:
df_statistica.to_csv(os.path.join(r"C:\Users\Andrea\desktop\part_freq.csv"))

In [41]:
if "morph_tags.csv" in os.getcwd():
    !del "morph_tags.csv" 

In [42]:
if "statistica.csv" in os.getcwd():
    !del "statistica.csv"

## Student texts
Now let's take some example student texts and extract tagsets and POS tagging using conllu API.

In [43]:
from conllu import parse, parse_tree

In [44]:
def parser(filename):
    """
    Yields a sentence from conllu tree with its tags

    """
    """
    >>> for i in parser('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedPsyEdu.conllu'):
      print(i)   
    TokenList<Музыка, звучит, отовсюду, независимо, от, нашего, желания, или, нежелания, слушать, ее, .>
    """
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
    tree = parse(data)
    for token in tree:
        yield token

In [45]:
def get_words(tree):
    """
    tree - generator of sentences (TokenLists) from conllu tree

    words, list is a list of all tokens we need from the tree
    size, int is a number of all words in the domain
    """
    words = []
    for sentence in tree:
        for token in sentence:
#             print(token)
            if token['form'] != '_' and token['upostag'] != '_' and token['upostag']!='NONLEX' and token['form'] not in r'[]\/':
                for wordform in token['form'].split(): # .lower()
                    words.append((wordform, token['lemma'], token['feats'], token['upostag']))
    size = len(words)
    return words, size

#### Добавляем словарь nplus

In [46]:
with open('nplus.csv', mode='r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    nplus = {(rows[0], rows[1]):rows[2] for rows in reader}

In [47]:
# nplus

#### Папки с студентическими текстами

In [48]:
stud_dir = r'C:\Users\Andrea\Desktop\stud_textVSscie_text\Student_texts_for_experiments\stud_txt'
low_lvl = os.path.join(stud_dir, 'Low Level')
reg_lvl = os.path.join(stud_dir, 'Regular Level')
low_prsd = os.path.join(stud_dir, 'conllu', 'Low_Level_Parsed')
reg_prsd = os.path.join(stud_dir, 'conllu', 'Regular_Level_Parsed')
ma_theses=r'C:\Users\Andrea\Desktop\stud_textVSscie_text\Student_texts_for_experiments\Fin_MA_theses_parsed'

In [49]:
def tagset_lemma(words):
    print('tagset being created...')
    with open('tagset.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['token', 'lemma', 'tagset', 'POS'])

        for word in words:
            if word[2]:
                tag_lst = []
                for tag in list(word[2].items()):
                    tag = '{}={}|'.format(tag[0], tag[1])
                    tag_lst.append(tag)

                tag_str = ''.join([str(elem) for elem in tag_lst])
                tag_str = tag_str[:-1]

                writer.writerow([word[0], word[1], tag_str, word[3]])
            # print(tag_str)
            else:
                tag_str = 'None'
                writer.writerow([word[0], word[1], tag_str, word[3]])
                
    df = pd.read_csv('tagset.csv')
    word_list = df.values.tolist()
    return word_list

In [50]:
def morph_error_catcher(words):
    mistakes = {}
    corrects = {}
    for i, word in enumerate(words):
        
        curD.execute("""SELECT unigram, lemm, morph, pos FROM
                (SELECT unigram, morph, lemma FROM unigrams) AS a JOIN
                (SELECT id_lemmas, id_pos, lemma AS lemm FROM lemmas) AS b ON lemma = id_lemmas JOIN pos ON b.id_pos = pos.id_pos
                WHERE unigram="{}" &&
                lemm="{}" &&
                morph="{}" && 
                pos="{}";""".format(word[0], word[1], word[2], word[3]))
        rowsD = curD.fetchall()
        if not rowsD:
            mistakes[i] = [word[0], word[1], word[2], word[3]]
        else:
            corrects[i] = [word[0], word[1], word[2], word[3]]
    return mistakes, corrects

In [51]:
from string import punctuation
punctuation += '«»—…“”–'
from nltk.corpus import stopwords
stops = stopwords.words('russian')

In [52]:
# def txt2conllu(filepath):
#     with open('{}'.format(filepath), 'r', encoding='utf-8') as f:
#         text = f.read()
    
#     temp = tempfile.TemporaryFile(mode='w+t', encoding='utf-8', suffix='.conllu')
    
# #     try:
#     temp.write(text)
#     return temp
# #     finally:
# #         temp.close()

In [53]:
corr_dir = r'C:\Users\Andrea\Desktop\corrected_files'

In [54]:
from colorama import Fore, Style 

In [55]:
import re
m = re.search('(?<=abc)def', 'abcdef')
m.group(0)

'def'

In [56]:
reg = re.compile(".+[a-z]+.+")
if reg.search("<<st>>"):
    print("reg")

reg


In [57]:
numbers = re.compile("[0-9]")
# latins = re.compile("[a-zA-Z]")
latins = re.compile("([a-zA-Z]+\W+)|(\W+[a-zA-Z]+)|(\W+[a-zA-Z]\W+)|([a-zA-Z]+)")
cyrillic = re.compile("([а-яА-ЯёЁ]+\W+)|(\W+[а-яА-ЯёЁ]+)|(\W+[а-яА-ЯёЁ]\W+)")
initial = re.compile("[а-яА-ЯёЁ]\.")
if latins.search("<f>"):
    print(True) 
else:
    print(False)

True


In [58]:
print(f'This is {Fore.RED}color{Style.RESET_ALL}!')

This is color!


In [59]:
def correctionX(filepath, corrected_files_directory):
    # the input must be a conllu format file
    
    filename=os.path.basename(os.path.normpath(filepath))
    # tagset creation
    tree = parser(filepath)
    words, size = get_words(tree)
    del tree   
    
    # correction process
    tot=0
    mists=0
    tagset = tagset_lemma(words)
    correction = []
    mistakes = morph_error_catcher(tagset)[0]
    # for idx in mistakes:
    for i, word in enumerate(tagset):
#         print(word)
        if i in mistakes and word[0].lower() not in punctuation and word[0].lower() not in stops and \
        (word[0].lower(), word[1].lower()) not in nplus and not numbers.match(word[0].lower()) and \
        not latins.search(word[0].lower()) and not cyrillic.search(word[0].lower()) and word[3] != 'PROPN':
            correction.append('++'+word[0]+'++')
            mists+=1
            tot+=1
        else:
            correction.append(word[0])
            tot+=1
    correction = ' '.join(correction)
    os.makedirs(corrected_files_directory, exist_ok=True)
    with open(os.path.join(corrected_files_directory, filename[:-6]+'txt'), 'w', encoding='utf-8') as fw:
        fw.write(correction)
    return print(correction, "\n\nCorrected words: %s" %mists, "\nMistake frequency: %s" %(mists/tot))

### Пример 1

In [60]:
correctionX(os.path.join(low_prsd, 'prs_EC12_B1_2421.conllu'), corr_dir)

tagset being created...
<B1 2421 > <Russian “н”=N > <о вас = у вас , ++нето++ = нету > Я хочу вам рассказать о ++дружба++ . По моему , это одних из самых важных вещи в жизни . ++Нада++ найти друзья с кем вы ++рассчитаивайте++ , ну быть ++рассчитавайная++ тоже главNое . Я ++согласин++ , что есть ++читири++ ++качествы++ нужно в друзья – честность , быть вежливый , юмор и общие интересы . Во - первых , друзья ++верют++ друг - другом . Например , это страшно , если вы ++сказайте++ секрет и ваш друг его ++сказает++ с них или неё друзья . Как муж и жена никогда сделаю чем то они будет ++виновать++ ++далшее++ , друзья ++вирит++ , что они будет <ъuдет > помогать если беда ++приэсходится++ . Самый хuчший друзья всегда открывают дверь - это называется ++открутонсть++ . Мой друзья мне ++рассчитавают++ , потому что я ++спросю++ " Всё хорошо ? " . Я всегда свободный если друг в кризис . Как много ++сказают++ , вы поступили из " ++приятель++ " до " друг " когда люди вам верит вы будете ним помогать 

### Пример 2

In [61]:
correctionX(os.path.join(low_prsd, 'prs_EC12-B1-0404.conllu'), corr_dir)

tagset being created...
﻿What is a friend ? Что такое друг ? Что такое друг ? ++Кажде++ человек что такое друг подругому . Ест ++луди++ кто читает друг человек кого они ++видет++ ++кажде++ день и разговаривает ++снеме++ . Но на другое ++старане++ ест то кто читает друг как семья и не ++знаит++ получается разговаривает ++кажде++ день . Я ++сегда++ читал что луча имет две или три хороши ++блиски++ друзья чем имет ++двадцат++ друзья но ++скем++ нет ++блиски++ ++отношени++ . Я пришёл в такую Эдеа потому что когда я был ++риёнок++ моя ++семя++ нога раз ++пережали++ в ++новие++ дом и мне была ++нада++ учитца в ++новие++ школа . В ++новие++ школа мне была ++нада++ найти новые ++друзя++ . Потому что я был новые студент , Меня ник- то не знал итак я питался ++подрузите++ с дети кто были добри , ++честни++ и смогли поддержать меня в ++трудние++ время . За что я питался ++друзить++ толка с хороши дети и не питался быт самы ++попюларне++ , я ишо имею ++блиски++ отношение с друзья каким я начинал +

### Пример 3 - средний уровень

In [62]:
correctionX(os.path.join(reg_prsd, 'prs_EC12_B1_4930.conllu'), corr_dir)

tagset being created...
<B1 4930 > <issues with linking letters > Что Такое Друг ? В ++нашой++ жизни мы встречаем разных людей . Каждой ++домает++ по разному , из других ++стациях++ , с других ++характерах++ . Но почему наших друзей выбрали с всех и ++которих++ мы ++зовом++ ++друзя++ . Обычно наши ++друзя++ подходят к ++нашому++ характеру . Мы часто не ++соримся++ но если и ++посоримся++ тогда ++прощаем++ друг друга . Я думаю друг должен ++помагать++ когда в ++плохои++ ++положеней++ , и когда трудно . На пример , я ++замыкнула++ ключе в машине и ++позвоню++ ей что бы она приехала по меня и она приедет . Когда в моей ++жизне++ будут трудности она будет слушать моей ++проблемии++ не ++перенесёть++ ++бругим++ о чём я говорила . Я знаю когда мне нужна с кем то , ++разгаваривать++ она ласково будет слушать . ++Эсли++ моя подруга часто обо мне ++говоритиие++ другим тогда я с ней не буду дружить . Нам должно ++наравиться++ в мести проводит время . В мое жизни часто бывали люди ++которий++ я +

### Пример 4 - средний уровень

In [63]:
correctionX(os.path.join(reg_prsd, 'prs_EC12-B1-0732.conllu'), corr_dir)

tagset being created...
Что такое друг ? Это не простой человек . Друг , это кто-то ++скем++ можно проводить время , с кем можно ++висилится++ , с кем можно говорить о хорошим и о трудным . Они там для вас когда они нужны и даже когда не обязательно ну они всё равно там для вас просто потому - что они заботливые . У меня есть одна подруга , Даша , которая является один из ++моех++ лучших ++груг++ . Она всё , что ++хорошого++ можно найти в ++други++ и ++болще++ . Я встретил Дашу в начале августа 2007 . Мы работали вместе в ++ракковой++ ++лаборитории++ . Мы ++тожи++ ходили в тоже самую школу ну до этого августа , мы были ++незнакомци++ . ++Даша++ была старший чем я и была в ++лаборитории++ уже один год когда я ++просоединился++ . Она мне всё показала как делать и мы проводили много времени ++вмести++ . Я был такой ++удевлён++ какая классная она была ! Её одежда , её ++вкуз++ в музыки , её интересные рассказы про как она ++танцавала++ и ++занималас++ тей-квон - до , всё мне было так интер

### MA Thesis

In [64]:
correctionX(os.path.join(ma_theses, 'Text1.conllu'), corr_dir)

tagset being created...
4 1 . ВВЕДЕНИЕ Передача политической речи оказывает сильное влияние на представление , которое читатель составляет себе о политике . Неточные и оторванные от контекста цитаты являются привычными не только в газетах , но и повсюду в средствах массовой информации . Например , подбор фактов , акцентирование только одной стороны явления и ++затушевывание++ другой , употребление оценочных метафор или эвфемистических ++перифразов++ при передаче новостей — все это сильно влияет на восприятие . Целью данной работы является определить способы , используемые журналистами при ++реферировании++ и цитировании политической речи : как называется оратор , какие глаголы и наречия употребляются и так далее . На этой основе можно создать представление о том , как выражается субъективный элемент , то есть точка зрения журналиста . Когда журналист пишет статью , ему надо выбрать подходящий стиль и жанр . После выбора того или иного жанра пишущий может бессознательно привнести те или

In [65]:
correctionX(os.path.join(ma_theses, 'Text2.conllu'), corr_dir)

tagset being created...
Введение Мир стал , и продолжает становиться глобальнее , и поэтому межкультурное общение и коммуникация значительно увеличились , и одновременно возросли потребности в знании иностранных языков . В Европейском союзе ++межкуль++ - ++турное++ общение считается богатством , ЕС хочет способствовать свободному движению граждан стран - членов , что предполагает владение иностранными языками . В Европе иностранные языки изучаются и преподаются больше и больше , но цели и методы обучения варьируются в разных странах . На обуче-ние иностранным языкам влияют ++многие++ факторы , и некоторые из них могут действовать только в определённой стране , поэтому можем обоснованно пред -полагать , что обучение иностранным языкам не может быть одинаково ++повсю++ - ду , а обучение нужно приспособить к обстановке . Многообразие есть и всегда будет , но всё-таки его важно опознавать и описывать так , чтобы сравнение стало возможным . ( Huttunen & Huttunen 1998 : 20 . ) От этих отправ

-------------------------------------------------

-------------------------------------------------

In [52]:
from ufal.udpipe import Model, Pipeline, ProcessingError
import os
import re
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

In [53]:
def make_conll_with_udpipe(text):
    model_path = 'C:\\Users\\Andrea\\russian-syntagrus-ud-2.4-190531.udpipe' # здесь указать путь к модели
    model = Model.load(model_path)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    return pipeline.process(text)

In [54]:
with open(os.path.join(low_lvl, 'EC12_B1_2421.txt'), 'r', encoding='utf-8') as txt:
        text = txt.read()
        
        text = re.sub('[«»]', r'"', text)
        text = re.sub('\*', '', text)
       
        x = make_conll_with_udpipe(text)
        with open(os.path.join(low_lvl, "prs_" + 'EC12_B1_2421.txt'.strip(".txt") + ".conllu"), 'w', encoding='utf-8') as fw:
            fw.write(x)

In [55]:
tree = parser(os.path.join(low_lvl, 'prs_EC12_B1_2421.conllu'))

In [56]:
words, size = get_words(tree)

In [57]:
words

[('<B1', '<B1', None, 'NUM'),
 ('2421', '2421', None, 'NUM'),
 ('>', '>', None, 'SYM'),
 ('<Russian', '<Russiaн-лат', OrderedDict([('Foreign', 'Yes')]), 'PROPN'),
 ('“н”=N', '“н”=N', OrderedDict([('Foreign', 'Yes')]), 'PROPN'),
 ('>', '>', None, 'SYM'),
 ('<о', '<о', OrderedDict([('Degree', 'Pos')]), 'ADV'),
 ('вас',
  'вы',
  OrderedDict([('Case', 'Acc'), ('Number', 'Plur'), ('Person', '2')]),
  'PRON'),
 ('=', '=', None, 'SYM'),
 ('у', 'у', None, 'ADP'),
 ('вас',
  'вы',
  OrderedDict([('Case', 'Gen'), ('Number', 'Plur'), ('Person', '2')]),
  'PRON'),
 (',', ',', None, 'PUNCT'),
 ('нето', 'нето', OrderedDict([('Case', 'Nom')]), 'PRON'),
 ('=', '=', None, 'SYM'),
 ('нету',
  'нет',
  OrderedDict([('Aspect', 'Imp'),
               ('Mood', 'Ind'),
               ('Number', 'Sing'),
               ('Person', '3'),
               ('Tense', 'Pres'),
               ('VerbForm', 'Fin'),
               ('Voice', 'Act')]),
  'VERB'),
 ('>', '>', None, 'SYM'),
 ('Я',
  'я',
  OrderedDict([('Ca

In [58]:
del tree

In [59]:
correctionX(words)

TypeError: correctionX() missing 1 required positional argument: 'corrected_files_directory'

-----------------------------------------------------------------

In [ ]:
words, size = get_words(tree)

In [ ]:
with open(os.path.join(low_lvl, 'EC12_B1_2421.txt'), 'r', encoding='utf-8') as txt:
        text = txt.read()
        
        text = re.sub('[«»]', r'"', text)
        text = re.sub('\*', '', text)
       
        x = make_conll_with_udpipe(text)
        with open(os.path.join(low_lvl, "prs_" + 'EC12_B1_2421.txt'.strip(".txt") + ".conllu"), 'w', encoding='utf-8') as fw:
            fw.write(x)

In [ ]:
tree = parser(os.path.join(low_lvl, 'prs_EC12_B1_2421.conllu'))

In [ ]:
words, size = get_words(tree)

In [ ]:
words

In [ ]:
del tree

In [ ]:
correctionX(words)

-----------------------------------------------------------------

In [ ]:
words, size = get_words(tree)

In [ ]:
words

In [ ]:
del tree

In [ ]:
correctionX(words)

-----------------------------------------------------------------

In [ ]:
dictn = {}

In [ ]:
keys = zip(df['tagset'], df['POS'])
for couple in keys:
    if couple in dictn:
        dictn[couple] += 1
    else:
        dictn[couple] = 1

In [ ]:
df_stat = pd.Series(dictn).reset_index()
df_stat.columns = ['tagset', 'POS', 'count'] 
df_stat[:5]

In [ ]:
df_stat['part_freq'] = df_stat['count'].apply(lambda x: x/df_stat['count'].sum(axis=0))

In [ ]:
df_stat

In [ ]:
df_statistica[:10]

In [ ]:
df_stat['count'].sum(axis=0)

In [ ]:
df_stat['x1000_freq'] = df_stat['count'].apply(lambda x: (x*1000)/df_stat['count'].sum(axis=0))

In [ ]:
df_stat = df_stat.sort_values(by='POS').reset_index(drop=True)

In [ ]:
df_stat[:10]

In [ ]:
df_statistica['x1000_freq'] = df_statistica['count'].apply(lambda x: (x*1000)/df_statistica['count'].sum(axis=0))

In [ ]:
df_statistica[:10]

## Comparing statistics
Let's now try to compare the tagset frequency for our corpus and for the student text we selected.

In [ ]:
corpus_kfreq = zip(df_statistica['tagset'], df_statistica['POS'], df_statistica['part_freq'])
corpus_dict = {(elem[0], elem[1]) : (elem[2],) for elem in corpus_kfreq}

In [ ]:
corpus_dict

In [ ]:
stud_kfreq = zip(df_stat['tagset'], df_stat['POS'], df_stat['part_freq'])
stud_dict = {(elem[0], elem[1]) : (elem[2],) for elem in stud_kfreq}

In [ ]:
stud_dict

In [ ]:
stat_dict = {}
for entry in stud_dict:
    if entry in corpus_dict:
        stat_dict[entry] = corpus_dict[entry] + (stud_dict[entry])

In [ ]:
stat_dict

In [ ]:
# d = pd.DataFrame.from_dict(stud_dict) 

In [ ]:
df_diff = pd.Series(stat_dict).reset_index()
df_diff.columns = ['tagset', 'POS', 'freq']
df_diff[['corpus_freq', 'stud_txt_freq']] = pd.DataFrame(df_diff['freq'].values.tolist(), index=df_diff.index)
del df_diff['freq']
df_diff

In [ ]:
df_diff['corpus_freq'].mean()

In [ ]:
ttest_ind(df_diff['corpus_freq'], df_diff['stud_txt_freq'])

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
# diff_dict = {}
# for elem in stud_dict:
#     if elem in corpus_dict:
#         diff = corpus_dict[elem] - stud_dict[elem]
#     diff_dict[elem] = diff

In [ ]:
df_diff = pd.Series(stat_dict).reset_index()
df_diff.columns = ['tagset', 'POS', 'freq']
df_diff[['corpus_freq', 'stud_txt_freq']] = pd.DataFrame(df_diff['freq'].values.tolist(), index=df_diff.index)
del df_diff['freq']
df_diff

In [ ]:
df_diff['corpus_freq'].mean()

In [ ]:
ttest_ind(df_diff['corpus_freq'], df_diff['stud_txt_freq'])

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
# diff_dict = {}
# for elem in stud_dict:
#     if elem in corpus_dict:
#         diff = corpus_dict[elem] - stud_dict[elem]
#     diff_dict[elem] = diff

In [ ]:
len(diff_dict)

In [ ]:
df_diff

In [ ]:
!del 'tagset.csv'

In [ ]:
# def tagset(words):
#     with open('tagset.csv', 'w', newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         writer.writerow(['token', 'tagset', 'POS'])

#         for word in words:
#             if word[1]:
#                 tag_lst = []
#                 for tag in list(word[1].items()):
#                     tag = '{}={}|'.format(tag[0], tag[1])
#                     tag_lst.append(tag)

#                 tag_str = ''.join([str(elem) for elem in tag_lst])
#                 tag_str = tag_str[:-1]

#                 writer.writerow([word[0], tag_str, word[2]])
#             # print(tag_str)
#             else:
#                 tag_str = 'None'
#                 writer.writerow([word[0], tag_str, word[2]]) 

In [ ]:
!pip install mysql-connector-python

In [ ]:
!c:\users\andrea\anaconda3\python.exe -m pip install --upgrade pip